In [71]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [159]:
import warnings
warnings.filterwarnings('ignore')

In [72]:
torch.cuda.is_available()

True

In [73]:
torch.backends.cudnn.enabled

True

In [74]:
bs,bptt = 64,70

In [3]:
import getarxivdata as gr

In [4]:
pwd

'/home/Maanas/fastai/courses/dl1'

In [7]:
gr.GetArXiv.update('data/Arxiv/all_arxiv.pkl')

0. Fetched 1000 abstracts published 2018-05-30 and earlier
Downloaded 0 new abstracts


True

## Language modeling

### Data

In [75]:
#PATH='/data2/datasets/part1/arxiv/'
PATH='data/Arxiv/'
df_mb = pd.read_csv(f'{PATH}arxiv.csv')
df_all = pd.read_pickle(f'{PATH}all_arxiv.pkl')

In [76]:
def get_txt(df):
    return '<CAT> ' + df.category.str.replace(r'[\.\-]','') + ' <SUMM> ' + df.summary + ' <TITLE> ' + df.title
df_mb['txt'] = get_txt(df_mb)
df_all['txt'] = get_txt(df_all)
n=len(df_all); n

49400

In [11]:
os.makedirs(f'{PATH}trn/yes', exist_ok=True)
os.makedirs(f'{PATH}val/yes', exist_ok=True)
os.makedirs(f'{PATH}trn/no', exist_ok=True)
os.makedirs(f'{PATH}val/no', exist_ok=True)
os.makedirs(f'{PATH}all/trn', exist_ok=True)
os.makedirs(f'{PATH}all/val', exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)

In [12]:
for (i,(_,r)) in enumerate(df_all.iterrows()):
    dset = 'trn' if random.random()>0.1 else 'val'
    open(f'{PATH}all/{dset}/{i}.txt', 'w').write(r['txt'])

In [13]:
for (i,(_,r)) in enumerate(df_mb.iterrows()):
    lbl = 'yes' if r.tweeted else 'no'
    dset = 'trn' if random.random()>0.1 else 'val'
    open(f'{PATH}{dset}/{lbl}/{i}.txt', 'w').write(r['txt'])

In [77]:
from spacy.symbols import ORTH

my_tok = spacy.load('en')

my_tok.tokenizer.add_special_case('<SUMM>', [{ORTH: '<SUMM>'}])
my_tok.tokenizer.add_special_case('<CAT>', [{ORTH: '<CAT>'}])
my_tok.tokenizer.add_special_case('<TITLE>', [{ORTH: '<TITLE>'}])
my_tok.tokenizer.add_special_case('<BR />', [{ORTH: '<BR />'}])
my_tok.tokenizer.add_special_case('<BR>', [{ORTH: '<BR>'}])

def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

In [78]:
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
FILES = dict(train='trn', validation='val', test='val')
md = LanguageModelData.from_text_files(f'{PATH}all/', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [79]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

tensor([[   23],
        [  136],
        [   25],
        [   10],
        [   19],
        [ 1046],
        [   11],
        [   97],
        [  244],
        [   48],
        [   40],
        [    0]], device='cuda:0')

In [10]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(2018, 17977, 1, 9047181)

In [145]:
TEXT.vocab.itos[0:12]

['<unk>', '<pad>', 'the', ',', '.', 'of', '-', 'and', 'a', 'to', 'in', 'we']

In [11]:
' '.join(md.trn_ds[0].text[:150])

'<cat> cscv <summ> in this report we present experimental results using \\emph{haussdorf - besicovich } fractal dimension for performing morphological galaxy classification . the fractal dimension is a topological , structural and spatial property that give us information about the space were an object lives . we have calculated the fractal dimension value of the main types of galaxies : ellipticals , spirals and irregulars ; and we use it as a feature for classifying them . also , we have performed an image analysis process in order to standardize the galaxy images , and we have used principal component analysis to obtain the main attributes in the images . galaxy classification was performed using machine learning algorithms : c4.5 , k - nearest neighbors , random forest and support vector machines . preliminary experimental results using 10-fold cross - validation show that fractal dimension helps to improve classification ,'

### Train

In [80]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [81]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [15]:
learner.fit(3e-3, 1, wds=1e-6)

A Jupyter Widget

epoch      trn_loss   val_loss                                
    0      4.500647   4.385295  



[4.3852952107029211]

In [16]:
learner.fit(3e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=2)

A Jupyter Widget

epoch      trn_loss   val_loss                                
    0      4.271859   4.178087  
    1      4.211471   4.085734                                
    2      4.109124   4.016017                                
    3      4.161693   4.035389                                
    4      4.071443   3.964629                                
    5      3.98708    3.909383                                
    6      3.954021   3.896353                                



[3.8963532595471451]

In [17]:
learner.save_encoder('adam2_enc')

In [18]:
pwd

'/home/Maanas/fastai/courses/dl1'

In [ ]:
learner.fit(3e-3, 10, wds=1e-6, cycle_len=5, cycle_save_name='adam3_10')

A Jupyter Widget

epoch      trn_loss   val_loss                                
    0      4.088305   3.970075  
    1      4.017814   3.923469                                
    4      3.870515   3.840727                                
    5      4.021998   3.918424                                
    6      3.980128   3.883922                                
    7      3.910758   3.844133                                
    8      3.888788   3.817781                                
    9      3.838794   3.809314                                
    10     3.989589   3.894977                                
    11     3.94296    3.861271                                
    17     3.861541   3.805772                                
    18     3.817054   3.780429                                
    19     3.799036   3.773918                                
    20     3.968256   3.858518                                
    21     3.907943   3.833841                                
    22     3.866019   

KeyboardInterrupt: 

In [23]:
learner.save_encoder('adam3_10_enc')
learner.save('adam3_10')

In [22]:
#Loadning previously trained model
learner.load('adam3_10_cyc_4')
# or learner.load('adam3_10')

In [15]:
#check validation loss of latest trained model Just for varification 
learner.fit(3e-3, 1, wds=1e-6)

A Jupyter Widget

epoch      trn_loss   val_loss                                
    0      3.972777   3.864416  



[3.8644163727905672]

In [24]:
#lets try training few epocs
learner.fit(3e-3, 2, wds=1e-6, cycle_len=5, cycle_save_name='adam3_5')

A Jupyter Widget

epoch      trn_loss   val_loss                                
    0      3.948816   3.849694  
    1      3.907296   3.822264                                
    2      3.833257   3.784781                                
    3      3.77593    3.760739                                
    4      3.756781   3.755007                                
    5      3.938771   3.83904                                 
    6      3.894877   3.813601                                
    7      3.838      3.777862                                
    8      3.773967   3.753273                                
    9      3.740126   3.748196                                



[3.7481960498006406]

In [25]:
learner.save_encoder('adam3_5_enc')

In [82]:
learner.save('adam3_5')
#learner.load('adam3_5')

In [86]:
learner.fit(3e-3, 8, wds=1e-6, cycle_len=10, cycle_save_name='adam3_5')

A Jupyter Widget

[ 0.       3.70587  3.61666]                                  
[ 1.       3.71738  3.61174]                                  
[ 2.       3.68606  3.59661]                                  
[ 3.       3.65407  3.5742 ]                                  
[ 4.       3.62901  3.55795]                                  
[ 5.       3.59921  3.53632]                                  
[ 6.       3.58401  3.52149]                                  
[ 7.       3.55126  3.50797]                                  
[ 8.       3.52965  3.50178]                                  
[ 9.       3.52336  3.49997]                                  
[ 10.        3.7109    3.60817]                               
[ 11.        3.69879   3.60047]                               
[ 12.        3.6735    3.58623]                               
[ 13.        3.64365   3.56568]                               
[ 14.        3.6099    3.54776]                               
[ 15.        3.58244   3.52829]                        

In [87]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20, cycle_save_name='adam3_20')

A Jupyter Widget

[ 0.       3.47841  3.4751 ]                                  
[ 1.       3.69717  3.57883]                                  
[ 2.       3.68267  3.57793]                                  
[ 3.       3.66797  3.57299]                                  
[ 4.       3.66805  3.56847]                                  
[ 5.       3.63489  3.56238]                                  
[ 6.       3.62479  3.54928]                                  
[ 7.       3.60663  3.53879]                                  
[ 8.       3.59124  3.53175]                                  
[ 9.       3.58617  3.52009]                                  
[ 10.        3.56924   3.51174]                               
[ 11.        3.5509    3.49974]                               
[ 12.        3.51595   3.49008]                               
[ 13.        3.50939   3.48222]                               
[ 14.        3.48886   3.47952]                               
[ 15.        3.4676    3.47311]                        

In [89]:
learner.save_encoder('adam3_20_enc')

In [88]:
learner.save('adam3_20')

### Test

In [83]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [84]:
m=learner.model

In [160]:
s="""<CAT> cscv <SUMM> algorithms that"""

In [155]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0).unsqueeze(0))

    m[0].bs=bs

In [172]:
sample_model(m,"<CAT> csni <SUMM> algorithms that",100)

...are able to achieve a high throughput and low latency , are a promising solution for the next generation of wireless networks . however , the performance of these algorithms is limited by the number of users and the number of users . in this paper , we propose a novel approach to the problem of scheduling the mobile users in a network of mobile devices . we propose a novel algorithm that is based on the concept of a distributed algorithm that can be used to solve the problem of scheduling the mobile devices in a network . the 

In [173]:
sample_model(m,"<CAT> cscv <SUMM> algorithms that",200)

...are able to detect and detect objects in images are a challenging task . in this paper , we propose a novel method for detecting the presence of a background image in a scene . the proposed method is based on the idea of a novel method for detecting the presence of a background image . the proposed method is based on the idea of a novel method for detecting the presence of a background image . the proposed method is evaluated on a set of images from the wild ( lfw ) dataset . the results show that the proposed method outperforms the state - of - the - art methods in terms of the detection accuracy and the accuracy of the detection . <title> a novel method for detecting salient objects in images <eos> 

In [163]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> on ")

...the use of deep learning for image classification <eos> 

In [164]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> on ")

...the performance of a wireless network with a single - hop    network <eos> 

In [165]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> towards ")

...a deep learning approach to image segmentation <eos> 

In [174]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> towards ")

...a new approach to wireless networks <eos> 

### Sentiment

In [167]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

In [168]:
class ArxivDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['yes', 'no']:
            for fname in glob(os.path.join(path, label, '*.txt')):
                with open(fname, 'r') as f: text = f.readline()
                examples.append(data.Example.fromlist([text, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data',
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [169]:
ARX_LABEL = data.Field(sequential=False)

In [171]:
type(ARX_LABEL)

torchtext.data.field.Field

In [117]:
md2 = TextData.from_splits(PATH, splits, bs)

In [9]:
#            dropout=0.3, dropouti=0.4, wdrop=0.3, dropoute=0.05, dropouth=0.2)

In [143]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_at_6(preds,targs):
    precision, recall, _ = precision_recall_curve(targs==2, preds[:,2])
    print(recall[precision>=0.6][0])
    return recall[precision>=0.6][0]

In [156]:
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.65, wdrop=0.5, dropoute=0.1, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.clip=25.

In [157]:
m3.load_encoder(f'adam3_20_enc')
lrs=np.array([1e-4,1e-3,1e-2])

In [158]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

A Jupyter Widget

[ 0.       0.47654  0.44322  0.78525]                         



A Jupyter Widget

[ 0.       0.43033  0.40192  0.80087]                        



In [159]:
m3.fit(lrs, 2, metrics=[accuracy], cycle_len=4, cycle_save_name='imdb2')

A Jupyter Widget

[ 0.       0.42236  0.39006  0.8194 ]                        
[ 1.       0.39477  0.37063  0.82086]                        
[ 2.       0.39389  0.37082  0.82449]                        
[ 3.       0.40728  0.36999  0.82195]                        
[ 4.       0.39308  0.3675   0.81977]                        
[ 5.       0.38662  0.36737  0.8234 ]                        
[ 6.       0.39259  0.36512  0.82486]                        
[ 7.       0.38047  0.36538  0.82522]                        



In [160]:
prec_at_6(*m3.predict_with_targs())

0.659305993691


0.65930599369085174

In [161]:
m3.fit(lrs, 4, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

A Jupyter Widget

[ 0.       0.38752  0.36351  0.82486]                        
[ 1.       0.38664  0.36123  0.82558]                        
[ 2.       0.3904   0.36098  0.82486]                        
[ 3.       0.37319  0.36144  0.82486]                        
[ 4.       0.38074  0.36334  0.82595]                        
[ 5.       0.36405  0.3594   0.82413]                        
[ 6.       0.38781  0.35914  0.82522]                        
[ 7.       0.37722  0.357    0.82631]                        



In [162]:
prec_at_6(*m3.predict_with_targs())

0.695583596215


0.69558359621451105